In [1]:
import torch
import torchvision.models as model
import torch.nn as nn
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader, Subset
from torchvision import transforms, datasets
import os
import glob
from tqdm import tqdm
from PIL import Image
import random
from sklearn.model_selection import train_test_split
import numpy as np
import timm.models as tmodels
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

In [21]:
m = ['X']
frame = [5,10,15]
method = ['DFD','DF','F2F','FS','NS']
v=[10,30,50,100]

w,h=224,224

transform = transforms.Compose([
    transforms.Resize((w,h)),
    transforms.ToTensor()
])

# one 

In [28]:
testmodel=tmodels.xception(pretrained=False)
if model == 'R18':
    testmodel = tmodels.resnet18(pretrained=False)
num_ftrs = testmodel.fc.in_features
testmodel.fc = nn.Linear(num_ftrs,2)
testmodel.load_state_dict(torch.load(f'weights/X_DFD_50_5_L2_T/best.pth'))
testmodel.cuda()

test_path = f'/media/data2/rjsdn/DeepFakeDetection/test'
test_img = datasets.ImageFolder(test_path,transform=transform)

test_loader = DataLoader(test_img,batch_size=16,shuffle=False,num_workers=8)

test_loss=0
acc = 0
count=0
p,r=torch.tensor([]),torch.tensor([])
testmodel.eval()
with torch.no_grad():
    for x,y in tqdm(test_loader):
        count+=1
        x=x.cuda()
        y=y.cuda()
        output = testmodel(x)
        pred=output.argmax(dim=1)
        p=torch.cat([p,pred.to("cpu")])
        r=torch.cat([r,y.to("cpu")])
        acc += pred.eq(y.view_as(pred)).sum().item()
    acc /= len(test_img)
    print("ACC : %.5f"%(acc*100))

print('#'*20)

100%|██████████| 250/250 [00:07<00:00, 31.89it/s]

ACC : 75.37500
####################


# DFD/frame/video

In [22]:
ans = {
                'DFD':[],
                'DF':[],
                'F2F':[],
                'FS':[],
                'NS':[]
            }

In [23]:
for model in m:
    for i in v:
        for f in frame:
            
            for me in method:
                print(model,i,f,me)
                testmodel=tmodels.xception(pretrained=False)
                if model == 'R18':
                    testmodel = tmodels.resnet18(pretrained=False)
                num_ftrs = testmodel.fc.in_features
                testmodel.fc = nn.Linear(num_ftrs,2)
                testmodel.load_state_dict(torch.load(f'weights/{model}_{me}_{i}_{f}_L2_T/best.pth'))
                testmodel.cuda()

                for j in range(1,6):
                    test_path = f'/media/data2/rjsdn/zoom/dataset/test{j}'
                    test_img = datasets.ImageFolder(test_path,transform=transform)

                    test_loader = DataLoader(test_img,batch_size=16,shuffle=False,num_workers=8)

                    test_loss=0
                    acc = 0
                    count=0
                    p,r=torch.tensor([]),torch.tensor([])
                    testmodel.eval()
                    with torch.no_grad():
                        for x,y in tqdm(test_loader):
                            count+=1
                            x=x.cuda()
                            y=y.cuda()
                            output = testmodel(x)
                            pred=output.argmax(dim=1)
                            p=torch.cat([p,pred.to("cpu")])
                            r=torch.cat([r,y.to("cpu")])
                            acc += pred.eq(y.view_as(pred)).sum().item()
                        acc /= len(test_img)
                        print("ACC : %.5f"%(acc*100))
                    
                    ans[me].append(acc*100)
                ans[me].append(' ')

                print('#'*20)

X 10 5 DFD


100%|██████████| 250/250 [00:07<00:00, 31.85it/s]


ACC : 77.02500


100%|██████████| 250/250 [00:07<00:00, 31.36it/s]


ACC : 77.05000


...

ACC : 96.77500


100%|██████████| 250/250 [00:08<00:00, 31.09it/s]


ACC : 97.37500


100%|██████████| 250/250 [00:07<00:00, 31.37it/s]

ACC : 98.47500
####################


In [24]:
import pandas as pd

In [25]:
pd.DataFrame(ans).to_csv('./X_res.csv',index=False)

# DFD,FS, ...

In [3]:
for model in m:
    for i in k:
        print(model,i)
        testmodel=tmodels.xception(pretrained=False)
        if model == 'R18':
            testmodel = tmodels.resnet18(pretrained=False)
        num_ftrs = testmodel.fc.in_features
        testmodel.fc = nn.Linear(num_ftrs,2)
        testmodel.load_state_dict(torch.load(f'weights/{model}_{i}_L2/best.pth'))
        testmodel.cuda()

        for j in range(1,6):
            test_path = f'/media/data2/rjsdn/zoom/dataset/test{j}'
            test_img = datasets.ImageFolder(test_path,transform=transform)

            test_loader = DataLoader(test_img,batch_size=16,shuffle=False,num_workers=8)

            test_loss=0
            acc = 0
            count=0
            p,r=torch.tensor([]),torch.tensor([])
            testmodel.eval()
            with torch.no_grad():
                for x,y in tqdm(test_loader):
                    count+=1
                    x=x.cuda()
                    y=y.cuda()
                    output = testmodel(x)
                    pred=output.argmax(dim=1)
                    p=torch.cat([p,pred.to("cpu")])
                    r=torch.cat([r,y.to("cpu")])
                    acc += pred.eq(y.view_as(pred)).sum().item()
                acc /= len(test_img)
                print("ACC : %.5f"%(acc*100))

        print('#'*20)

R18 DFD


100%|██████████| 250/250 [00:04<00:00, 58.86it/s]


ACC : 45.40000


100%|██████████| 250/250 [00:04<00:00, 53.48it/s]


ACC : 48.75000


...

ACC : 52.10000


100%|██████████| 250/250 [00:04<00:00, 57.34it/s]


ACC : 54.00000


100%|██████████| 250/250 [00:03<00:00, 65.55it/s]

ACC : 53.52500
####################


# compression

In [6]:
for model in m:
    for frame in f:
        for i in k:
            print(model,frame,i)
            testmodel=tmodels.xception(pretrained=False)
            if model == 'R18':
                testmodel = tmodels.resnet18(pretrained=False)
            num_ftrs = testmodel.fc.in_features
            testmodel.fc = nn.Linear(num_ftrs,2)
            testmodel.load_state_dict(torch.load(f'weights/{model}_t{frame}_{i}_low/best.pth'))
            testmodel.cuda()

            for j in range(1,6):
                test_path = f'/media/data2/rjsdn/zoom/dataset/compression/test{j}'
                test_img = datasets.ImageFolder(test_path,transform=transform)

                test_loader = DataLoader(test_img,batch_size=16,shuffle=False,num_workers=8)

                test_loss=0
                acc = 0
                count=0
                p,r=torch.tensor([]),torch.tensor([])
                testmodel.eval()
                with torch.no_grad():
                    for x,y in tqdm(test_loader):
                        count+=1
                        x=x.cuda()
                        y=y.cuda()
                        output = testmodel(x)
                        pred=output.argmax(dim=1)
                        p=torch.cat([p,pred.to("cpu")])
                        r=torch.cat([r,y.to("cpu")])
                        acc += pred.eq(y.view_as(pred)).sum().item()
                    acc /= len(test_img)
                    print("ACC : %.5f"%(acc*100))

            print('#'*20)

X 6 5


100%|██████████| 250/250 [00:24<00:00, 10.17it/s]


ACC : 98.35000


100%|██████████| 250/250 [00:20<00:00, 12.03it/s]


ACC : 99.30000


...

ACC : 95.57500


100%|██████████| 250/250 [00:13<00:00, 18.03it/s]


ACC : 95.50000


100%|██████████| 250/250 [00:13<00:00, 17.99it/s]

ACC : 97.70000
####################


#  NonCompression

In [4]:
for model in m:
    for frame in f:
        for i in k:
            print(model,frame,i)
            testmodel=tmodels.xception(pretrained=False)
            if model == 'R18':
                testmodel = tmodels.resnet18(pretrained=False)
            num_ftrs = testmodel.fc.in_features
            testmodel.fc = nn.Linear(num_ftrs,2)
            testmodel.load_state_dict(torch.load(f'weights/{model}_t{frame}_{i}_L2/best.pth'))
            testmodel.cuda()

            for j in range(1,8):
                test_path = f'/media/data2/rjsdn/zoom/dataset/test{j}'
                test_img = datasets.ImageFolder(test_path,transform=transform)

                test_loader = DataLoader(test_img,batch_size=16,shuffle=False,num_workers=8)

                test_loss=0
                acc = 0
                count=0
                p,r=torch.tensor([]),torch.tensor([])
                testmodel.eval()
                with torch.no_grad():
                    for x,y in tqdm(test_loader):
                        count+=1
                        x=x.cuda()
                        y=y.cuda()
                        output = testmodel(x)
                        pred=output.argmax(dim=1)
                        p=torch.cat([p,pred.to("cpu")])
                        r=torch.cat([r,y.to("cpu")])
                        acc += pred.eq(y.view_as(pred)).sum().item()
                    acc /= len(test_img)
                    print("ACC : %.5f"%(acc*100))

            print('#'*20)

X 7 5


100%|██████████| 250/250 [00:07<00:00, 31.35it/s]


ACC : 56.10000


100%|██████████| 250/250 [00:07<00:00, 31.41it/s]


ACC : 58.67500


...

ACC : 57.72500


100%|██████████| 250/250 [00:07<00:00, 31.79it/s]


ACC : 96.79439


100%|██████████| 250/250 [00:08<00:00, 30.86it/s]

ACC : 96.30000
####################


In [25]:
# testmodel=tmodels.xception(pretrained=False)
testmodel=tmodels.resnet18(pretrained=False)
num_ftrs = testmodel.fc.in_features
testmodel.fc = nn.Linear(num_ftrs,2)
testmodel.load_state_dict(torch.load(f'weights/R18P_D//best.pth'))
testmodel.cuda()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (act1): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act1): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act2): ReLU(inplace=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  

In [8]:
a = [1,1,0,0]
b = ['R18P_D','R18D_P','X_D','X_P_D']
for i in range(4):
    
    testmodel=tmodels.xception(pretrained=False)
    if a[i]==1: testmodel=tmodels.resnet18(pretrained=False)
    num_ftrs = testmodel.fc.in_features
    testmodel.fc = nn.Linear(num_ftrs,2)
    testmodel.load_state_dict(torch.load(f'weights/{b[i]}/best.pth'))
    testmodel.cuda()
    for j in range(1,6):
        test_path = f'/media/data2/rjsdn/zoom/dataset/test{j}'
        test_img = datasets.ImageFolder(test_path,transform=transform)

        test_loader = DataLoader(test_img,batch_size=16,shuffle=False,num_workers=8)

        test_loss=0
        acc = 0
        count=0
        p,r=torch.tensor([]),torch.tensor([])
        testmodel.eval()
        with torch.no_grad():
            for x,y in tqdm(test_loader):
                count+=1
                x=x.cuda()
                y=y.cuda()
                output = testmodel(x)
                pred=output.argmax(dim=1)
                pred = pred ^ 1
                p=torch.cat([p,pred.to("cpu")])
                r=torch.cat([r,y.to("cpu")])
                acc += pred.eq(y.view_as(pred)).sum().item()
            acc /= len(test_img)
            print("ACC : %.5f"%(acc*100))
#             report = classification_report(r,p,target_names=['fake','real'],digits=4)
            report = confusion_matrix(r,p)
            print(report)
            print('fake',report[:,0].sum(),'real',report[:,1].sum())
#             with open('report.txt','a') as f:
#                 f.write(report)



100%|██████████| 250/250 [00:03<00:00, 69.12it/s] 


ACC : 50.60000
[[1874  126]
 [1850  150]]
fake 3724 real 276


100%|██████████| 250/250 [00:04<00:00, 58.63it/s] 


ACC : 50.12500
[[1856  144]
 [1851  149]]
fake 3707 real 293


100%|██████████| 250/250 [00:03<00:00, 65.46it/s] 


...

ACC : 52.82500
[[1758  242]
 [1645  355]]
fake 3403 real 597


100%|██████████| 250/250 [00:06<00:00, 35.71it/s]


ACC : 50.82500
[[1784  216]
 [1751  249]]
fake 3535 real 465


100%|██████████| 250/250 [00:06<00:00, 35.80it/s]

ACC : 53.40000
[[1766  234]
 [1630  370]]
fake 3396 real 604
